## Building a Chatbot 


In this video we'll go over an eg of how to design and implement an LLM Powered chatbot. This chatbot will be able to have a conversation and remember 
previous interactions. 

Note that this chtbotthat we build will only use the language model to have a conversation. 

There are several other related concepts that you may be looking for  Like:

- Conversational RAG: Enable a chatbot experience over an external source of data.
- Agents: Buid a chatbot that can take actions 


In [1]:
import os 
import pydantic.v1
from dotenv import load_dotenv 
load_dotenv() 

groq_api_key= os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ce340e8a3b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ce340eb4550>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:

from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Ankit i am a Machine learning & Generative A.I Engineer")])


AIMessage(content="Hi Ankit, it's nice to meet you!\n\nThat's a fascinating field!  As a large language model, I'm constantly learning about the amazing things people are doing with machine learning and generative AI.\n\nWhat kind of projects are you working on these days? I'd love to hear more about your work.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 28, 'total_tokens': 100, 'completion_time': 0.130909091, 'prompt_time': 0.000140399, 'queue_time': 0.013099000000000001, 'total_time': 0.13104949}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24fd2348-d07d-4075-a96f-1414e37adc5a-0', usage_metadata={'input_tokens': 28, 'output_tokens': 72, 'total_tokens': 100})

In [15]:
from langchain_core.messages import AIMessage

model.invoke(
    [
      HumanMessage(content="Hi, my name is Ankit i am a Machine learning & Generative A.I Engineer"),
      AIMessage(content="Hello Ankit! It's nice to meet you. \n\n a ML & Gen A.I Engineer, what kindoff project you are working on , am always eager to know about A.I"),
      HumanMessage(content="Hey what's my name and what do i do ? ")    

    ]
)


# Important thing here is, it is able to remember previous message context 


AIMessage(content="You said your name is Ankit, and that you are a Machine Learning & Generative AI Engineer!  \n\nIs there anything else you'd like me to know about you or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 92, 'total_tokens': 138, 'completion_time': 0.083636364, 'prompt_time': 0.002596733, 'queue_time': 0.010668476999999999, 'total_time': 0.086233097}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-82b37e82-e2d0-4680-86dd-a6edd833e963-0', usage_metadata={'input_tokens': 92, 'output_tokens': 46, 'total_tokens': 138})

###  Messag History 

This is one of the imp things in Langchain 

- We can use a Message history class to wrap our model and mke it stateful, This will keep track of inputs and outputs of the model, and store them in some   
datastore. Further interactions will then load those messages and pass them into the chain as part of the input. 
Let's see how to use this 


pip install langchain-community 

In [5]:
# !pip install langchain-community 


- ChatMessageHistory: Yeh ek class hai jo har session ke messages ko track karegi.

- BaseChatMessageHistory: Yeh ek base (mool) class hai jo define karti hai ki kaise chat history handle ki jaye. Sab classes jo chat history    
ke sath kaam karti hain, usse inherit karti hain.    

- RunnableWithMessageHistory: Yeh class model ke saath message history ko combine karti hai, taki humara model purane messages  
 ka use karke naye responses de sake.  

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# how to make sure one session is different then another session 

store = {}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        # Whenever we are storing session id we are initializing chatmessage history 
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history) # for our model to make sure our model interacts with chat history 


RunnableWithMessageHistory ka use hota hai humare model ke sath chat history ko integrate karne ke liye.   
model: Yeh aapka AI model hai jo responses generate karega.   
get_session_history: Yeh function model ko help karta hai session ke specific messages ko retrieve karne mein.   
Iska matlab yeh hai ki jab user koi naya message bhejega, toh model ko us session ka pura context milega purane   
 messages se, aur model better respond kar payega.

In [7]:
config = {"configurable": {"session_id":"chat1"}}

In [19]:
response =with_message_history.invoke(
    [HumanMessage(content="what's my name?")],
    config=config
)

response.content

'Hi Ankit!  \n\nI remember you told me your name is Ankit. 😊  \n\nIs there anything I can help you with? \n'

In [20]:
config2 = {"configurable": {"session_id":"chat2"}}

response =with_message_history.invoke(
    [HumanMessage(content="what's my name?")],
    config=config2
)

response.content

'Your name is John.  \n\nIs there anything else I can help you with? 😊 \n'

In [21]:
response =with_message_history.invoke(
    [HumanMessage(content="My name is John")],
    config=config2
)

response.content

"It seems we both share the name John!  That's interesting.  \n\nWhat can I do for you, John? 😄 \n"

In [22]:
response =with_message_history.invoke(
    [HumanMessage(content="What's my Name ?")],
    config=config2
)

response.content

'Your name is John.  \n\nIs there something specific you want to know about your name, John? 😊 \n'

In [23]:
response =with_message_history.invoke(
    [HumanMessage(content="What's my Name ?")],
    config=config
)

response.content

"Your name is Ankit!  \n\nI'm glad I could remember it. 😊  \n\nHow can I help you today?\n"

In [24]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content="As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'm happy to know! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 15, 'total_tokens': 56, 'completion_time': 0.074545455, 'prompt_time': 7.653e-05, 'queue_time': 0.013819178, 'total_time': 0.074621985}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f9db0e87-28f0-4b0d-a94e-a742b7113fcd-0', usage_metadata={'input_tokens': 15, 'output_tokens': 41, 'total_tokens': 56}), HumanMessage(content="What's my Name ?", additional_kwargs={}, response_metadata={}), AIMessage(content="Since I'm just a language model, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to te

### Prompt Templates 


Prompt templates help to turn raw user information into a format that the LLM can work with. 
In this case raw user input is just a message. which we are passing to the LLM. Let's now make that a bit more complicated.   
First Let's add in a system message with some custo instructions .

Next we'll add in more input besided just the messages.




In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are helpfull assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
    
)

chain = prompt|model


In [29]:
## Whatever messages we gonna give , we must give in Key Value Pair
## with Key "messages"

chain.invoke(
{
    "messages": [ HumanMessage(content="Hi my name is Ankit") ]
}
)



AIMessage(content="Hello Ankit, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 31, 'total_tokens': 58, 'completion_time': 0.049090909, 'prompt_time': 0.000356658, 'queue_time': 0.01394047, 'total_time': 0.049447567}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-775b36c8-31c6-4555-a325-c8b086695a2b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58})

In [31]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

config2 = {
    "configurable":{"session_id": "chat3"}      
          }

In [32]:
response = with_message_history.invoke(

[ HumanMessage(content="Hi my name is Ankit") ],
config=config2
)

response

AIMessage(content="Hello Ankit! It's nice to meet you.  \n\nHow can I help you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.001707964, 'queue_time': 0.04864019, 'total_time': 0.048980691}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1453f0b9-2cfe-4962-9326-ac69800df7ea-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [33]:
response = with_message_history.invoke(

[ HumanMessage(content="what is my Name ?") ],
config=config2
)

response

AIMessage(content='Your name is Ankit.  \n\nYou told me at the beginning of our conversation! 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 70, 'total_tokens': 93, 'completion_time': 0.041818182, 'prompt_time': 0.002063843, 'queue_time': 0.011061026000000002, 'total_time': 0.043882025}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f8b29ab-eb53-4f51-8597-9a3e4d49ecc5-0', usage_metadata={'input_tokens': 70, 'output_tokens': 23, 'total_tokens': 93})

### Let's add more complexity 



In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are helpfull assistant. Answer all the questions to the best of your ability in {language}"),
        
        MessagesPlaceholder(variable_name="messages")
    ]
    
)

chain = prompt|model



In [37]:
response = chain.invoke(

 {
     "messages":[HumanMessage(content="Hi My name is Ankit ")],
     "language":"Hindi"
 }
)


response.content 

'नमस्ते अनकिट!  \n\nआपसे बात करना अच्छा लग रहा है। \n\nमुझे आपकी सारी प्रश्नों के उत्तर देने में खुशी होगी। \n\nआप क्या जानना चाहते हैं? \n\n'

Let's now wrap this  more complicated chain in a Message History class. This time, because there are multiple keys in the input,     
we need to specify correct key to use to save the chat history.

In [38]:
with_message_history = RunnableWithMessageHistory(

chain, 
get_session_history,
input_messages_key="messages"
)

In [42]:

config3 = {
    "configurable":{"session_id": "chat4"}      
          }


response = with_message_history.invoke(
 
 {'messages':[HumanMessage(content="Hi i am Ankit")], "language":"hindi" },
 config=config3

)

response.content

'नमस्ते Ankit!  \n\nआपका नाम सुनकर अच्छा लगा। \n\nक्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

In [43]:
response = with_message_history.invoke(
 
 {'messages':[HumanMessage(content="what is my Name")], "language":"hindi" },
 config=config3

)

response.content

'आपका नाम Ankit है।  😊 \n'

### Managing the Conversation History 


One important concept to understand when building chatbots is how to manage conversation history. if left unmanaged, the list of messages will 

grow unbounded and potentially overflow the context window of the LLM.

Therefore, it is important to add a step that limits the size of messages you are passing in. 

'trim_messages' helper to reduce how many messages we're sending to the model. 
The trimmer allows us to specify how many token we want to keep along with other parameters like if we want to always keep the system messages and whether to allow partial messages. 






In [53]:
from langchain_core.messages import SystemMessage,trim_messages

# trim messages : it will help to reduce how many messages we're sending to Model 
# trimmer allows us to specify how many token we want to keep along with other parameters 

trimmer=trim_messages(

max_tokens = 75, 
strategy="last",  # last menas last convo pe focus krega
token_counter=model, # kitne tokens honge , model use hoga count krne smjho
include_system=True,
allow_partial=False,
start_on = "human"
)


# Define a set of messages
messages = [
    SystemMessage(content="Hello! How can I help you today?"),
    HumanMessage(content="Hi! What's the result of 2+2?"),
    SystemMessage(content="The result of 2+2 is 4."),
    HumanMessage(content="Great, thanks!"),
    SystemMessage(content="You're welcome! Goodbye."),
    HumanMessage(content="Bye!")
]


trimmer.invoke(messages)

[SystemMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi! What's the result of 2+2?", additional_kwargs={}, response_metadata={}),
 SystemMessage(content='The result of 2+2 is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Great, thanks!', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="You're welcome! Goodbye.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Bye!', additional_kwargs={}, response_metadata={})]

In [54]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages= itemgetter("messages")|trimmer) | prompt | model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What question i have asked")],
    "language":"English"
    }
)

response.content

'You asked: "What\'s the result of 2+2?" \n'